<div class="alert alert-block alert-success">
<b>Imported libraries</b>
</div>

In [1]:
import numpy as np
import cvxpy as cp
import sympy as sym
import autograd.numpy as np

from sympy import * 
from autograd import grad
from scipy.optimize import minimize

<div class="alert alert-block alert-info">
<b>Symbols</b>
</div>

In [2]:
x, y, l, L, k = sym.symbols('x y l L k')

<div class="alert alert-block alert-info">
<b>Objective and constraint functions</b>
</div>

In [3]:
f = 6*x**2 + 3*x - 11*y**2
g = 5*x**2 + 2*x - 10*y**2
lam = f - L * g


display(f)
display(g)
display(lam)

6*x**2 + 3*x - 11*y**2

5*x**2 + 2*x - 10*y**2

-L*(5*x**2 + 2*x - 10*y**2) + 6*x**2 + 3*x - 11*y**2

<div class="alert alert-block alert-info">
<b>Compute KKT conditions</b>
</div>

In [4]:
gradL = [sym.diff(L,c) for c in [x,y]] # gradient of Lagrangian w.r.t. (x,y)
KKT_eqs = gradL + [g]
KKT_eqs

[0, 0, 5*x**2 + 2*x - 10*y**2]

<div class="alert alert-block alert-info">
<b>Potential minimizers</b>
</div>

In [5]:
stationary_points = sym.solve(KKT_eqs, [x, y, L], dict = True) # solve the KKT equations
stationary_points 

[{x: -sqrt(50*y**2 + 1)/5 - 1/5}, {x: sqrt(50*y**2 + 1)/5 - 1/5}]

<div class="alert alert-block alert-info">
<b>Use the objective function for each point to determine the minimum</b>
</div>

In [6]:
[f.subs(p) for p in stationary_points]

[-11*y**2 - 3*sqrt(50*y**2 + 1)/5 + 6*(-sqrt(50*y**2 + 1)/5 - 1/5)**2 - 3/5,
 -11*y**2 + 3*sqrt(50*y**2 + 1)/5 + 6*(sqrt(50*y**2 + 1)/5 - 1/5)**2 - 3/5]

<div class="alert alert-block alert-info">
<b>Simple way</b>
</div>

In [7]:
# For equality constrains
objective = Eq(3*x**2-2*x*l, 0)
constrain  = Eq(x**2+y**2, 1)

# One equality constraint.
print('Objective:')
display(objective)

print('Subject to:')
display(constrain)

print('Possible results:')
sol = solve([Eq(3*x**2-2*x*l ,0),
      Eq(3*y**2-2*y*l, 0),
      Eq(x**2+y**2, 1)], [x,y,l])

display(sol)

Objective:


Eq(-2*l*x + 3*x**2, 0)

Subject to:


Eq(x**2 + y**2, 1)

Possible results:


[(-1, 0, -3/2),
 (0, -1, -3/2),
 (0, 1, 3/2),
 (1, 0, 3/2),
 (-sqrt(2)/2, -sqrt(2)/2, -3*sqrt(2)/4),
 (sqrt(2)/2, sqrt(2)/2, 3*sqrt(2)/4)]

In [8]:
# For inequality constrains
objective = Eq(3*x**2+2*x*l**2,0)
constrain  = Eq(x**2+y**2+k**2,1)
constrain_convert  = Eq(x**2+y**2-1,0)

# One equality constraint.
print('Objective:')
display(objective)

# Can convert it into an equality constrain by introducing a slack variable, k
print('Subject to with slack variable:')
display(constrain)

print('Converted')
display(constrain_convert)

print('Possible results:')
sol = solve([Eq(3*x**2+2*x*l**2,0),
      Eq(3*y**2+2*y*l**2,0),
      Eq(x**2+y**2+k**2,1),
      Eq(x**2*l+y**2*l-l,0)], [x,y,l,k])

display(sol)

Objective:


Eq(2*l**2*x + 3*x**2, 0)

Subject to with slack variable:


Eq(k**2 + x**2 + y**2, 1)

Converted


Eq(x**2 + y**2 - 1, 0)

Possible results:


[(-1, 0, -sqrt(6)/2, 0),
 (-1, 0, sqrt(6)/2, 0),
 (0, -1, -sqrt(6)/2, 0),
 (0, -1, sqrt(6)/2, 0),
 (0, 0, 0, -1),
 (0, 0, 0, 1),
 (0, 1, -sqrt(6)*I/2, 0),
 (0, 1, sqrt(6)*I/2, 0),
 (1, 0, -sqrt(6)*I/2, 0),
 (1, 0, sqrt(6)*I/2, 0),
 (-sqrt(2)/2, -sqrt(2)/2, -2**(1/4)*sqrt(3)/2, 0),
 (-sqrt(2)/2, -sqrt(2)/2, 2**(1/4)*sqrt(3)/2, 0),
 (sqrt(2)/2, sqrt(2)/2, -2**(1/4)*sqrt(3)*I/2, 0),
 (sqrt(2)/2, sqrt(2)/2, 2**(1/4)*sqrt(3)*I/2, 0)]

<div class="alert alert-block alert-info">
<b>With Numpy</b>
</div>

In [9]:
# Sign = - 1 for Maximum and no Sign for Minimum
# 'eq' For equality

def objective(X):
    x, y, z = X
    return x**2 + y**2 + z**2

def constraint1(X):
    x, y, z = X
    return 2 * x - y + z - 3

sol = minimize(objective, [1, -0.5, 0.5], constraints = {'type': 'eq', 'fun': constraint1})
sol

     fun: 1.5
     jac: array([ 2.00000001, -0.99999999,  1.00000001])
 message: 'Optimization terminated successfully.'
    nfev: 5
     nit: 1
    njev: 1
  status: 0
 success: True
       x: array([ 1. , -0.5,  0.5])

In [10]:
# To find x array

def F(L):
    'Augmented Lagrange function'
    x, y, z, _lambda = L
    return objective([x, y, z]) - _lambda * constraint1([x, y, z])

# Gradients of the Lagrange function
dfdL = grad(F, 0)

# Find L that returns all zeros in this function.
def obj(L):
    x, y, z, _lambda = L
    dFdx, dFdy, dFdz, dFdlam = dfdL(L)
    return [dFdx, dFdy, dFdz, constraint1([x, y, z])]

from scipy.optimize import fsolve
x, y, z, _lam = fsolve(obj, [0.0, 0.0, 0.0, 1.0])
print(f'The answer is at {x, y, z}')

The answer is at (1.0, -0.5, 0.5)


In [11]:
# Sign = - 1 for Maximum and no Sign for Minimum
# 'ineq' For inequality

def objective(x):
    sign = - 1
    x1 = x[0]
    x2 = x[1]
    return sign * (3 * x1 + 5 * x2)

def constraint1(x):
    return 18.0 - 3* x[0] - 2 * x[1]

x0 = [0, 0]
b1 = (0, 4)
b2 = (0, 6)
bnds = (b1, b2)
cons = {'type': 'ineq', 'fun': constraint1}
sol = minimize(objective, x0, method = 'SLSQP', bounds = bnds, constraints = cons)
print(sol)

     fun: -35.99999999999172
     jac: array([-3., -5.])
 message: 'Optimization terminated successfully.'
    nfev: 12
     nit: 3
    njev: 3
  status: 0
 success: True
       x: array([2., 6.])


<div class="alert alert-block alert-info">
<b>Matrix problem with CVXPY</b>
</div>

In [12]:
# Problem data
m = 20
n = 20
np.random.seed(1)
A = np.random.randn(m, n)
b = np.random.randn(m)

# The problem
x = cp.Variable(n)
objective = cp.Minimize(cp.sum_squares(A@x - b))
constraints = [0 <= x, x <= 1]
problem = cp.Problem(objective, constraints)

# The optimal objective value is returned by `problem.solve()`
result = problem.solve()
# The optimal value for x is stored in `x.value`
print(x.value)
# The optimal Lagrange multiplier for a constraint is stored in `constraint.dual_value`
print(constraints[0].dual_value)

[ 3.82931579e-01  1.00000000e+00  6.26048287e-01 -1.35471555e-20
  1.12084817e-02 -6.04767434e-21 -3.16509064e-21  1.00167352e-01
  8.65011050e-01  5.06454750e-01  2.80702223e-01  5.83132330e-01
 -2.68829843e-20  3.07922421e-01 -7.80470713e-21  1.88492328e-01
  4.09306358e-01  1.41151229e-01  3.33278444e-02  5.15754509e-02]
[ 0.          0.          0.          4.48830236  0.          5.21912347
  4.48729148  0.          0.          0.          0.          0.
  5.65938809  0.         10.47822484  0.          0.          0.
  0.          0.        ]
